# SMT

In [468]:
from itertools import combinations
from z3 import *

In [469]:
#Caricamento dati

file = open("./Instances/inst01.dat", "r")
lines = file.readlines()
file.close()
m = int(lines[0].rstrip(''))
n = int(lines[1].rstrip(''))
load = [int(x) for x in lines[2].split(' ')]
size = [int(x) for x in lines[3].split(' ')]
D = [[int(x) for x in line.rstrip().split(' ') ]for line in lines[4:]]
o = n+1
print(m)
print(n)
print(load)
print(size)
for i in range(n+1):
    for j in range(n+1):
        print(D[i][j], end = " ")
    print()

#Definizione solver
s = Solver()

2
6
[15, 10]
[3, 2, 6, 5, 4, 4]
0 3 4 5 6 6 2 
3 0 1 4 5 7 3 
4 1 0 5 6 6 4 
4 4 5 0 3 3 2 
6 7 8 3 0 2 4 
6 7 8 3 2 0 4 
2 3 4 3 4 4 0 


In [470]:
#Utils
def at_least_one_np(bool_vars):
    return Or(bool_vars)

def at_most_one_np(bool_vars):
    return And([Not(And(pair[0], pair[1])) for pair in combinations(bool_vars, 2)])

def exactly_one_np(bool_vars):
    return And(at_least_one_np(bool_vars), at_most_one_np(bool_vars))

In [471]:
#Decision variables
assignment = [[Bool(f'assignment_{i}_{j}') for j in range(m)] for i in range(n)]
tour = [[Int(f'tour_{i}_{j}') for j in range(n+2)] for i in range(m)]

In [472]:
#Constraints

# Each item is assigned to exactly one courier
for i in range(n):
    s.add(exactly_one_np([assignment[i][j] for j in range(m)]))

# Each courier carries at most its capacity
for j in range(m):
    s.add(sum([If(assignment[i][j], size[i], 0) for i in range(n)]) <= load[j])

# Each courier starts from the depot
for c in range(m):
    s.add(And(tour[c][0] == o, tour[c][n+1] == o))


# Each courier visits only assigned items
for c in range(m):
    for i in range(n):
        s.add(Implies(assignment[i][c], tour[c][i+1] == i+1))

for c in range(m):
    for i in range(n):
        s.add(Implies(Not(assignment[i][c]), tour[c][i+1] == o))

        

# Objective function


In [473]:

if s.check() == sat:
    model = s.model()
    print("sat")
    for i in range(n):
        for j in range(m):
            print(model[assignment[i][j]], end = " ")
        print()
    for i in range(m):
        for j in range(n+2):
            print(model[tour[i][j]], end = " ")
        print()
else:
    print("unsat")

sat
True False 
True False 
False True 
True False 
False True 
True False 
7 1 2 7 4 7 6 7 
7 7 7 3 7 5 7 7 
